In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import shutil
from glob import glob

In [2]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img 

/home/neelr/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
cwd = os.getcwd()
source = cwd + "/data/complete"
dest_train = cwd + "/data/train"
dest_val = cwd + "/data/val"
labels = os.listdir(source)
print(len(labels))

8


In [4]:
class_names = glob(source)
class_names = sorted(class_names)

In [4]:
for label in labels:
    files = os.listdir(source + '/' + label)
    os.mkdir(dest_train + '/' + label)
    os.mkdir(dest_val + '/' + label)
    for f in files:
        if np.random.rand(1) > 0.2:
            shutil.copy(source + '/' + label + '/' + f, dest_train + '/' + label + '/' + f)
        else:
            shutil.copy(source + '/' + label + '/' + f, dest_val + '/' + label + '/' + f)

FileExistsError: [Errno 17] File exists: '/media/neelr/Personal/Neel/EDrive/BITS/SEM VIII/NNFL/Lab/LabIBM/data/train/bed'

In [5]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense

In [6]:
model = Sequential()

model.add(Conv2D(64, (3, 3), input_shape=(180, 180, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(8))
model.add(Activation('sigmoid'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['categorical_accuracy'])

In [7]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 178, 178, 64)      1792      
_________________________________________________________________
activation_1 (Activation)    (None, 178, 178, 64)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 89, 89, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 87, 87, 64)        36928     
_________________________________________________________________
activation_2 (Activation)    (None, 87, 87, 64)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 43, 43, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 41, 41, 128)       73856     
__________

In [8]:
train_datagen = ImageDataGenerator(
    rescale = 1/255,
    shear_range = 0.2, 
    zoom_range = 0.2,
    horizontal_flip = True)

val_datagen = ImageDataGenerator(
    rescale = 1/255, 
    shear_range = 0.2, 
    zoom_range = 0.2,
    horizontal_flip = True)

In [9]:
train_generator = train_datagen.flow_from_directory(
    dest_train,
    target_size = (180, 180),
    batch_size = 16,
    class_mode = 'categorical')

validation_generator = val_datagen.flow_from_directory(
    dest_val,
    target_size = (180,180),
    batch_size = 16,
    class_mode = 'categorical')

Found 3048 images belonging to 8 classes.
Found 784 images belonging to 8 classes.


In [10]:
name_id_map = dict(zip(class_names, range(len(class_names))))

In [11]:
model.fit_generator(
    train_generator,
    steps_per_epoch = 250,
    epochs = 20,
    validation_data = validation_generator,
    validation_steps = 100
    )

Epoch 1/20
250/250 [==============================] - 494s 2s/step - loss: 1.6278 - categorical_accuracy: 0.4337 - val_loss: 1.1281 - val_categorical_accuracy: 0.6875
Epoch 2/20
250/250 [==============================] - 500s 2s/step - loss: 1.0344 - categorical_accuracy: 0.6745 - val_loss: 0.6391 - val_categorical_accuracy: 0.7812
Epoch 3/20
250/250 [==============================] - 501s 2s/step - loss: 0.7755 - categorical_accuracy: 0.7570 - val_loss: 0.5201 - val_categorical_accuracy: 0.8263
Epoch 4/20
250/250 [==============================] - 494s 2s/step - loss: 0.6671 - categorical_accuracy: 0.7945 - val_loss: 0.5530 - val_categorical_accuracy: 0.8281
Epoch 5/20
250/250 [==============================] - 492s 2s/step - loss: 0.5852 - categorical_accuracy: 0.8128 - val_loss: 0.5083 - val_categorical_accuracy: 0.8431
Epoch 6/20
250/250 [==============================] - 513s 2s/step - loss: 0.4994 - categorical_accuracy: 0.8347 - val_loss: 0.4565 - val_categorical_accuracy: 0.870

In [12]:
model.save('model1.h5')

In [13]:
print(train_generator.class_indices)

{'bed': 0, 'chair': 1, 'lamp': 2, 'shelf': 3, 'sofa': 4, 'stool': 5, 'table': 6, 'wardrobe': 7}
